## ODOC Public Inmate Data


This notebook is intended as a start for research of the ODOC data.  To use this notebook, follow the setup instructiion found [here](https://github.com/codefortulsa/odoc-parse). Then download the data published [here](http://doc.publishpath.com/odoc-public-inmate-data).  Unzip the file and place the files in a subdirectory called 'data'.

The set of files includes a ReadMe.txt which describes the files and their fixed formats. The sections of this notebook show description of each file and how to import it into pandas dataframes.  NOTE: the widths variables differ slighty from the description to handle some difference in the data.

In [121]:
import pandas as pd

## Schedule A - Profile Data Layout
 ```
 =======================================================
 Name                            Null?    Type
 ------------------------------- -------- ----
 DOC_NUM                         NOT NULL NUMBER(10)
 LAST_NAME                                VARCHAR2(30)
 FIRST_NAME                               VARCHAR2(30)
 MIDDLE_NAME                              VARCHAR2(30)
 SUFFIX                                   VARCHAR2(5)
 LAST_MOVE_DATE                           DATE 'DD-MMM-YY' (9)
 FACILITY                                 VARCHAR2(40)
 BIRTH_DATE                               DATE 'DD-MMM-YY' (9)
 SEX                                      VARCHAR2(1)
 RACE                                     VARCHAR2(40)
 HAIR                                     VARCHAR2(40)
 HEIGHT_FT                                VARCHAR2(2)
 HEIGHT_IN                                VARCHAR2(2)
 WEIGHT                                   VARCHAR2(4)
 EYE                                      VARCHAR2(40)
 STATUS                                   VARCHAR2(10)
 ```

In [129]:
file = 'data/Vendor_Profile_Sample_Text.DAT'

# uncomment this line to use the full dataset
file = 'data/Vendor_Profile_Extract_Text.DAT'

names = [
    "DOC_NUM"       
    ,"LAST_NAME"     
    ,"FIRST_NAME"    
    ,"MIDDLE_NAME"   
    ,"SUFFIX"        
    ,"LAST_MOVE_DATE"
    ,"FACILITY"      
    ,"BIRTH_DATE"    
    ,"SEX"           
    ,"RACE"          
    ,"HAIR"          
    ,"HEIGHT_FT"     
    ,"HEIGHT_IN"     
    ,"WEIGHT"        
    ,"EYE"           
    ,"STATUS"
]
widths = [
    11,
    30,
    30,
    30,
    5,
    9,
    40,
    9,
    1,
    40,
    40,
    2,
    2,
    4,
    40,10
]

profile_df = pd.read_fwf(file, 
    header=None,
    widths=widths,
    names=names)
                 
profile_df.set_index("DOC_NUM", inplace=True)
profile_df.dropna(subset=['LAST_NAME'], inplace=True)

profile_df

,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,LAST_MOVE_DATE,FACILITY,BIRTH_DATE,SEX,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS
DOC_NUM,,,,,,,,,,,,,,,
8359,MESLES,ARDELL,NaN,NaN,03-MAR-86,INACTIVE,06-APR-87,M,NaN,NaN,NaN,NaN,NaN,NaN,Inactive
10337,SAWYER,FRANK,N,NaN,12-JUN-91,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
12556,GOFORTH,PAMELA,B,NaN,13-NOV-89,INACTIVE,06-FEB-51,M,NATIVE AMERICAN,BROWN,5.0,9.0,116.0,N/R,Inactive
20413,KNIGHTON,RAYMOND,J,NaN,18-APR-80,INACTIVE,20-AUG-48,M,BLACK,BROWN,NaN,NaN,NaN,BROWN,Inactive
22574,STOCKTON,MELVIN,H,NaN,02-JAN-97,INACTIVE,07-DEC-45,M,WHITE,BROWN,5.0,10.0,150.0,GRAY,Inactive
22663,MACON,EUGENE,NaN,NaN,30-MAR-98,INACTIVE,26-JUL-39,M,BLACK,UNK,5.0,10.0,NaN,N/R,Inactive
27399,GARCIA,JOSE,L,NaN,11-JUL-16,NORTH FORK CORRECTIONAL CENTER ESCAPE,11-SEP-43,M,HISPANIC,BLACK,5.0,8.0,165.0,BROWN,Active
27486,HIXON,BILLY,J,NaN,24-SEP-88,INACTIVE,18-DEC-45,M,WHITE,BROWN,5.0,10.0,160.0,N/R,Inactive
31179,GREENWOOD,BUFORD,NaN,NaN,20-NOV-80,INACTIVE,13-MAR-17,M,BLACK,BLACK,5.0,9.0,207.0,N/R,Inactive


## Schedule C - Sentence Data Layout
```
=======================================================
Incarcerated_Term_In_Years = 9999 indicates a death sen
tence

Incarcerated_Term_In_Years = 8888 indicates a life with
out parole sentence

Incarcerated_Term_In_Years = 7777 indicates a life sent
ence
=======================================================
 Name                            Null?    Type
 ------------------------------- -------- ----
 DOC_NUM                         NOT NULL NUMBER(10)
 STATUTE_CODE                    NOT NULL VARCHAR2(40)
 SENTENCING_COUNTY                        VARCHAR2(40)
 JS_DATE                                  DATE 'YYYYMMDD' (8)
 CRF_NUMBER                               VARCHAR2(40)
 INCARCERATED_TERM_IN_YEARS               NUMBER(10,2)
 PROBATION_TERM_IN_YEARS                  NUMBER(10,2)
 ```

In [123]:
from datetime import datetime

file = 'data/Vendor_sentence_Sample_Text.dat'

# uncomment this line to use the full dataset
file = 'data/Vendor_sentence_Extract_Text.dat'


names =[
    "DOC_NUM",                   
    "STATUTE_CODE",              
    "SENTENCING_COUNTY",         
    "JS_DATE",                   
    "CRF_NUMBER",                
    "INCARCERATED_TERM_IN_YEARS",
    "PROBATION_TERM_IN_YEARS"
]

widths = [
    10,
    40,
    40,
    10,
    40,
    13,
    13
]

sentence_df = pd.read_fwf(file, 
    header=None,
    widths=widths,
    names=names)

def make_date (dataDate):
    try:
        date = dataDate.split('-')
        year = date[2]
        if int(year) > 50:
            date[2]="19"+year
        else:
            date[2]="20"+year

        fixedDate = "-".join(date)
        dt = datetime.strptime(fixedDate, "%d-%b-%Y")
        return dt
    except:
        print(dataDate)
        return


sentence_data = sentence_df.copy()
# sentence_df.dropna(subset=['JS_DATE'], inplace=True)  
sentence_df['JS_DATE'] = sentence_df['JS_DATE'].apply(make_date)
sentence_df.sort_values(sentence_df.columns[3], inplace=True)
sentence_data.set_index('DOC_NUM', inplace=True)
sentence_df



nan
nan


,DOC_NUM,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS
1089888,65632,921-1283,MUSKOGEE COUNTY COURT,1951-02-16,CF-2016-1251,NaN,776.96
99,5228,821-701.7,MUSKOGEE COUNTY COURT,1951-04-20,1-0,7777.0,NaN
97,5228,821-701.7,MUSKOGEE COUNTY COURT,1951-04-20,51-8950,0.0,NaN
96,5228,821-701.7,MUSKOGEE COUNTY COURT,1951-04-20,51-8950,7777.0,NaN
436740,23956,463-2-402,TULSA COUNTY COURT,1951-05-10,98-4797,2.0,NaN
13339,9247,021-1713,OKLAHOMA COUNTY COURT,1952-02-28,2000-5902,2.0,NaN
25889,9817,547-11-902,OKLAHOMA COUNTY COURT,1952-03-20,19-771,3.0,NaN
10821,9081,521-701.7,OKLAHOMA COUNTY COURT,1952-04-16,20-746,7777.0,NaN
118,5347,921-701.7,TULSA COUNTY COURT,1952-04-24,14-836,NaN,NaN
981456,58847,059-1512,CLEVELAND COUNTY COURT,1952-05-17,2014-2214,NaN,5.00


In [127]:
import oscn
import re


counties = oscn.counties

def parse_county(dataCounty):
    if "JURISDICTION" in dataCounty:
#         print(dataCounty)
        return None
    county = dataCounty.split(" ")[0].lower()
    if county in counties:
        return county
#     print(dataCounty)
    return None

def parse_year(crf_number):
    split_number = re.split("-| ", crf_number)
    for i in range(2):
        try:
            maybe_year = int(split_number[i])
            if (maybe_year < 2021 and maybe_year > 1900):
                return split_number[i]
            if (maybe_year < 2021 and maybe_year > 1900):
                return split_number[i]
            if (maybe_year < 100 and maybe_year > 20):
                return "19" + split_number[i]
            if (maybe_year < 9 and maybe_year >= 0):
                return "200" + split_number[i]
            if (maybe_year < 21 and maybe_year >= 10):
                return "20" + split_number[i]
        except:
            continue
#     print(crf_number)
    return None
    
def parse_number(crf_number):
    split_number = crf_number.split("-")
    return split_number[len(split_number)-1]


sentence_data['oscn_county'] = sentence_data['SENTENCING_COUNTY'].apply(parse_county)
sentence_data.dropna(subset=['oscn_county'], inplace=True)  
sentence_data['oscn_year'] = sentence_data['CRF_NUMBER'].apply(parse_year)
sentence_data['oscn_number'] = sentence_data['CRF_NUMBER'].apply(parse_number)

sentence_data

TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
OHIO JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
MISSOURI JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
NEW JERSEY JURISDICTION
ARIZONA JURISDICTION
NORTH CAROLINA JURISDICTION
NORTH CAROLINA JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
NEW MEXICO JURISDICTION
KANSAS JURISDICTION
VIRGINIA JURISDICTION
FLORIDA JURISDICTION
FLORIDA JURISDICTION
FLORIDA JURISDICTION
FLORIDA JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEX

KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
MISSOURI JURISDICTION
KANSAS JURISDICTION
MISSOURI JURISDICTION
CALIFORNIA JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TENNESSEE JURISDICTION
OREGON JURISDICTION
WASHINGTON JURISDICTION
WASHINGTON JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
NEW MEXICO JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
IDAHO JURISDICTION
IDAHO JURISDICTION
IOWA JURISDICTION
IOWA JURISDICTION
MICHIGAN JURISDICTION
KANSAS JURISDICTION
SOUTH CAROLINA JURISDICTION
SOUTH CAROLINA JURISDICTION
TEXAS JURISDICTION
FLORIDA JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
TENNESSEE JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
NEBRASKA JURISDICTION
NEW MEXICO JURISDICTION
TEXAS JURISDICTION
CALIFORNIA JURISDICTION
GEORGIA JURISDICTION
GEORGIA JURISDICTION
GEORGIA JURISDICTION
G

TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
ILLINOIS JURISDICTION
ILLINOIS JURISDICTION
ALABAMA JURISDICTION
TEXAS JURISDICTION
NEW MEXICO JURISDICTION
OHIO JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
COLORADO JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KAN

TEXAS JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
IOWA JURISDICTION
MICHIGAN JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
WISCONSIN JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
FLORIDA JURISDICTION
FLORIDA JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
VIRGINIA JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARIZONA JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
CALIFORNIA JURISDICTION
NEW MEXICO JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
MISSOURI JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
NEW MEXICO JURISDICTION
NEW MEXICO JURISDICTION
KANSAS JURISDICTION

NEW MEXICO JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
OREGON JURISDICTION
COLORADO JURISDICTION
COLORADO JURISDICTION
COLORADO JURISDICTION
KANSAS JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
MONTANA JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
ARIZONA JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURIS

...

ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
CALIFORNIA JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
ARIZONA JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
UTAH JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
COLORADO JURISDICTION
OHIO JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
FLORIDA JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
ILLINOIS JURISDICTION
KANSAS JURISDICTION
COLORADO JURISDICTION
COLORADO JURISDICTION
TEXAS JURISDICTION
LOUISIANA JURISDICTION
COLORADO JURISDICTION
MISSOURI JURISDICTION
MISSO

TEXAS JURISDICTION
IOWA JURISDICTION
WYOMING JURISDICTION
WYOMING JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
MISSISSIPPI JURISDICTION
MISSOURI JURISDICTION
ARKANSAS JURISDICTION
TENNESSEE JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
MINNESOTA JURISDICTION
MINNESOTA JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
TEXAS JURISDICTION
LOUISIANA JURISDICTION
LOUISIANA JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
OHIO JURISDICTION
ARKANSAS JURISDICTION
LOUISIANA JURISDICTION
FLORIDA JURISDICTION
TEXAS JURISDICTION
CALIFORNIA JURISDICT

KANSAS JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
NORTH CAROLINA JURISDICTION
ARKANSAS JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
NEW MEXICO JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
WASHINGTON JURISDICTION
WASHINGTON JURISDICTION
MISSOURI JURISDICTION
COLORADO JURISDICTION
KANSAS JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
PENNSYLVANIA JURISDICTION
PENNSYLVANIA JURISDICTION
PENNSYLVANIA JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
KANSAS JURISDICTION
NORTH CAROLINA JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTIO

DELAWARE JURISDICTION
DELAWARE JURISDICTION
ARKANSAS JURISDICTION
ARKANSAS JURISDICTION
ARIZONA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
INDIANA JURISDICTION
GEORGIA JURISDICTION
ARKANSAS JURISDICTION
COLORADO JURISDICTION
ARIZONA JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
MISSOURI JURISDICTION
KANSAS JURISDICTION
TEXAS JURISDICTION
WASHINGTON JURISDICTION
COLORADO JURISDICTION
TEXAS JURISDICTION
MINNESOTA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
CALIFORNIA JURISDICTION
ARIZONA JURISDICTION
CALIFORNIA JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
WASHINGTON JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
CALIFORNIA JURISDICTION
OREGON JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
MICHIGAN JURISDICTION
MICHIGAN JURISDICTION
TEXAS JURISDICTION
MISSOURI JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
TEXAS JURISDICTION
OREGON JURISDICTION
MISSOURI JURISDICTION


,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,oscn_county,oscn_year,oscn_number
DOC_NUM,,,,,,,,,
835,921-1720,POTTAWATOMIE COUNTY COURT,13-MAR-13,2010-300,NaN,1.50,pottawatomie,2010,300
1033,721-701.7,BRYAN COUNTY COURT,13-APR-20,1-0,7777.00,NaN,bryan,2001,0
1033,721-701.7,BRYAN COUNTY COURT,14-DEC-62,20-15645,NaN,40.00,bryan,2020,15645
2041,321-1731,TULSA COUNTY COURT,19-APR-77,77-240,NaN,3.00,tulsa,1977,240
2257,421-741,TULSA COUNTY COURT,03-MAR-64,20-393,7777.00,NaN,tulsa,2020,393
2257,447-11-902,TULSA COUNTY COURT,24-OCT-94,94-4167,NaN,1.00,tulsa,1994,4167
2266,321-801.A,TULSA COUNTY COURT,22-APR-64,2-513,36.00,NaN,tulsa,2002,513
2266,321-801.A,TULSA COUNTY COURT,22-APR-64,2-513,12.00,NaN,tulsa,2002,513
2739,963-2-402,OKLAHOMA COUNTY COURT,09-NOV-72,72-491,5.00,NaN,oklahoma,1972,491


In [125]:
joined_df = sentence_data.set_index('DOC_NUM').join(profile_df.set_index('DOC_NUM'))

# joined_df.dropna(subset=['LAST_NAME'], inplace=True)

joined_df

,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,...,FACILITY,BIRTH_DATE,SEX,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS
DOC_NUM,,,,,,,,,,,,,,,,,,,,,
835,921-1720,POTTAWATOMIE COUNTY COURT,13-MAR-13,2010-300,NaN,1.50,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1033,721-701.7,BRYAN COUNTY COURT,13-APR-20,1-0,7777.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1033,721-701.7,BRYAN COUNTY COURT,14-DEC-62,20-15645,NaN,40.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2041,321-1731,TULSA COUNTY COURT,19-APR-77,77-240,NaN,3.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2257,421-741,TULSA COUNTY COURT,03-MAR-64,20-393,7777.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2257,447-11-902,TULSA COUNTY COURT,24-OCT-94,94-4167,NaN,1.00,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2266,321-801.A,TULSA COUNTY COURT,22-APR-64,2-513,36.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2266,321-801.A,TULSA COUNTY COURT,22-APR-64,2-513,12.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2739,963-2-402,OKLAHOMA COUNTY COURT,09-NOV-72,72-491,5.00,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
joined_df.dropna(subset=['LAST_NAME'], inplace=True)

joined_df

,STATUTE_CODE,SENTENCING_COUNTY,JS_DATE,CRF_NUMBER,INCARCERATED_TERM_IN_YEARS,PROBATION_TERM_IN_YEARS,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,...,FACILITY,BIRTH_DATE,SEX,RACE,HAIR,HEIGHT_FT,HEIGHT_IN,WEIGHT,EYE,STATUS
DOC_NUM,,,,,,,,,,,,,,,,,,,,,
8359,163-2-401,OKLAHOMA COUNTY COURT,10-JUN-87,87-615,5.00,NaN,MESLES,ARDELL,NaN,NaN,...,INACTIVE,06-APR-87,M,NaN,NaN,NaN,NaN,NaN,NaN,Inactive
10337,021-1720,MCCURTAIN COUNTY COURT,04-OCT-79,79-167,1.00,2.00,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,121-1435,MCCURTAIN COUNTY COURT,05-OCT-79,77-134,2.00,NaN,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,221-1116,POTTAWATOMIE COUNTY COURT,05-SEP-79,79-220,NaN,2.00,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,421-1431,POTTAWATOMIE COUNTY COURT,03-JAN-92,91-197,10.00,NaN,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,421-1431,POTTAWATOMIE COUNTY COURT,03-JAN-92,91-197,NaN,10.00,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,4F-FELONY,POTTAWATOMIE COUNTY COURT,02-JAN-92,91-216,NaN,10.00,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,421-1435,POTTAWATOMIE COUNTY COURT,14-APR-98,97-388,20.00,NaN,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
10337,421-1720,CLEVELAND COUNTY COURT,03-JUN-91,91-546,NaN,2.00,SAWYER,FRANK,N,NaN,...,INACTIVE,01-JAN-00,M,WHITE,UNK,5.0,1.0,NaN,BLACK,Inactive
